# Leveraging PINNs For Multi-Dimensional Pricing Problems 
#### Author: JP Melo

This thesis is focused on the application of Physics-Informed Neural Networks (PINNs) to solve multi-dimensional pricing problems in finance. The equation we attempt to solve is described below.

### **Equation**
First, we define the scaled variables 
$$
  x_i \;=\; \ln\!\Bigl(\frac{S_i}{K}\Bigr),
  \quad 
  u(\tau, x_1, \dots, x_d) \;=\; \frac{V\bigl(t,S_1,\dots,S_d\bigr)}{K}
  \quad\text{with}\quad 
  \tau \;=\; T - t.
$$

Under risk-neutral pricing in backward time $\tau$, the function 
$u(\tau, x_1, \dots, x_d)$ satisfies the PDE
$$
\frac{\partial u}{\partial \tau} \;=\;
\frac{1}{2}\sum_{i=1}^d 
\sigma_i^{2}\!\left(
\frac{\partial^{2}u}{\partial x_i^{2}}
-\frac{\partial u}{\partial x_i}\right)
\;+\;
\frac{1}{2}\sum_{i=1}^d\sum_{j=1}^d
\sigma_i\sigma_j\rho_{ij}\,
\frac{\partial^{2}u}{\partial x_i\,\partial x_j}
\;+\;
r\sum_{i=1}^d\frac{\partial u}{\partial x_i}
\;-\;
r\,u .
$$

### **Boundary Conditions**

**Bottom boundary**  
For very small $S_i$ i.e. $x_i \to -\infty$, one commonly imposes (1 asset)
$$
    -\frac{\partial u}{\partial \tau} - ru \;=\; 0.
$$

For the multi-asset case, the k'th lower boundary condition is given by

$$
\frac{\partial u}{\partial \tau} \;=\;
\frac{1}{2}\sum_{\substack{i=1 \\ i\neq k}}^d 
\sigma_i^{2}\!\left(
\frac{\partial^{2}u}{\partial x_i^{2}}
-\frac{\partial u}{\partial x_i}\right)
\;+\;
\frac{1}{2}\sum_{\substack{i=1 \\ i\neq k}}^d
      \sum_{\substack{j=1 \\ j\neq k}}^d
\sigma_i\sigma_j\rho_{ij}\,
\frac{\partial^{2}u}{\partial x_i\,\partial x_j}
\;+\;
r\sum_{\substack{i=1 \\ i\neq k}}^d
\frac{\partial u}{\partial x_i}
\;-\;
r\,u .
$$

**Top boundary**  
For very large $S_i$ i.e. $x_i \to +\infty$, assume asymptotically linear behavior in $S_i$,
which translates to the following expression in the dimensionless case (1 asset):
$$
   \frac{\partial^2 u}{\partial x_i^2}-\frac{\partial u}{\partial x_i} = 0
$$

The generalization for the multi-asset case is straightforward is presented below:

$$
\frac{\partial u}{\partial \tau} \;=\;
\frac{1}{2}\sum_{\substack{i=1 \\ i\neq k}}^d 
\sigma_i^{2}\!\left(
\frac{\partial^{2}u}{\partial x_i^{2}}
-\frac{\partial u}{\partial x_i}\right)
\;+\;
\frac{1}{2}\sum_{\substack{i=1 \\ i\neq k}}^d
      \sum_{\substack{j=1 \\ j\neq k}}^d
\sigma_i\sigma_j\rho_{ij}\,
\frac{\partial^{2}u}{\partial x_i\,\partial x_j}
\;+\;
r\sum_{i=1}^d\frac{\partial u}{\partial x_i}
\;-\;
r\,u .
$$

### Imports

In [1]:
from derpinns.nn import *
from derpinns.utils import *
from derpinns.trainer import *
import torch
import kfac

/Users/josemelo/Desktop/master/tesis/codes/.conda/lib/python3.11/site-packages/kfac/base_preconditioner.py:15: UserWarning: NVIDIA Apex is not installed or was not installed with --cpp_ext. Falling back to PyTorch flatten and unflatten.
  from kfac.distributed import get_rank


## Parameters

In [2]:
# Global parameters
assets = 10

sampler = "Sobol"  # ["pseudo", "LHS", "Halton", "Hammersley", "Sobol"]:
device = torch.device("cpu")
dtype = torch.float32
nn_shape = "64x3"

# Define option valuation params
params = OptionParameters(
    n_assets=assets,
    tau=1.0,
    sigma=np.array([0.2] * assets),
    rho=np.eye(assets) + 0.25 * (np.ones((assets, assets)
                                         ) - np.eye(assets)),
    r=0.05,
    strike=100,
    payoff=payoff
)

# Build the net to be used
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)

## Other possible net models
# model = NNAnzats(n_layers=3, input_dim=assets+1,hidden_dim=64, output_dim=1).apply(weights_init).to(device)
# model = SPINN(n_layers=3, input_dim=assets+1, hidden_dim=32, output_dim=1).apply(weights_init).to(device)
# model = NNWithFourier(n_layers=3, input_dim=assets+1, hidden_dim=64, output_dim=1).apply(weights_init).to(device)

## Training

For training, different optimizers are used in order to get better accuracy as stated in [this article](https://arxiv.org/pdf/2402.01868).

### Adam Training

In [ ]:
# Set the training parameters
batch_size = 200
total_iter = 2_000
boundary_samples = 5_000
interior_samples = boundary_samples*assets*2
initial_samples = boundary_samples*assets*2

# Create dataset to traing over
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device)

# Set optimizer and training function
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


# Set the training function
closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\
    # .with_scheduler(scheduler)

trainer.train()

In [ ]:
results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device)['l2_rel_error']
print("L2 Error: ", results*100)

L2 Error:  15.175803676368767


In [ ]:
# plot the loss
state = trainer.closure.get_state()
plot_loss(state, nn_shape)

### LBFGS Training

In [ ]:
boundary_samples = 500
interior_samples = boundary_samples*assets*2
initial_samples = boundary_samples*assets*2

# We create new samples
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device)

optimizer = LBFGS(
    model.parameters(),
    max_eval=5_000,
    max_iter=1_000,
    line_search_fn="strong_wolfe",
)
batch_size = len(dataset) # we use all samples

closure = closure.with_dataset(
    dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})

trainer = trainer.with_optimizer(optimizer).with_training_step(closure)
trainer.train()

state = closure.get_state()
plot_loss(state, nn_shape, smooth=False)

LBFGS training:   1%|          | 28/5000 [01:05<2:58:29,  2.15s/it, Interior=0.004227, Boundary=0.063291, Initial=0.271236, Total=0.338754, Max Error=75.020752, L2 Error=0.115447] Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1040ce910>>
Traceback (most recent call last):
  File "/Users/josemelo/Desktop/master/tesis/codes/.conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
LBFGS training:   1%|          | 34/5000 [01:18<2:56:02,  2.13s/it, Interior=0.004066, Boundary=0.057666, Initial=0.257887, Total=0.319619, Max Error=67.365743, L2 Error=0.101571]

### NysNewtonCG Training

In [ ]:
# boundary_samples = 50
# interior_samples = boundary_samples*assets*2
# initial_samples = boundary_samples*assets*2

# dataset = SampledDataset(
#     params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device)
# batch_size = len(dataset)

# optimizer = NysNewtonCG(model.parameters(), line_search_fn='armijo', rank=60, mu=0.1, cg_tol=1e-5)

# closure = closure.with_dataset(
#     dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})

# trainer = trainer.with_optimizer(optimizer).with_epochs(10)
# trainer.train()

# state = closure.get_state()
# plot_loss(state, nn_shape)

In [ ]:
results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device)['l2_rel_error']
print("L2 Error: ", results*100)

L2 Error:  28.604071529529985
